In [1]:
!git clone --depth 1 --filter=blob:none --sparse https://github.com/koa-fin/sep.git
%cd sep
!git sparse-checkout set data

fatal: destination path 'sep' already exists and is not an empty directory.
/content/sep


In [2]:
!pip install -U g4f
!pip install peft==0.6.2
!pip install  bitsandbytes==0.42.0
!pip install datasets==2.14.7
!pip install tiktoken==0.6.0
!pip install transformers==4.34.1
!pip  install evaluate==0.4.1
!pip install trl==0.7.1
!pip install protobuf==4.25.2

In [3]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from g4f.client import Client
import g4f
import tiktoken
import re

import os, json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaForSequenceClassification, LlamaConfig
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
)
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
import sys

import torch
import torch.nn as nn
# import bitsandbytes as bnb
import transformers
import argparse
import warnings


from datasets import load_dataset
import json
from typing import List
from dataclasses import dataclass, field
from typing import Optional

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_int8_training
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, BitsAndBytesConfig
from transformers import LlamaTokenizer, LlamaConfig, LlamaForSequenceClassification, LlamaForCausalLM

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler



/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:

class OpenAILLM:
    def __init__(self):
        self.model = "gpt-3.5-turbo"

    def __call__(self, prompt):
     client = Client()
     completion = client.chat.completions.create(
         model=self.model,
         messages=[{"role": "user", "content": prompt}],
     )
     response = completion.choices[0].message.content
     return response

In [ ]:
SUMMARIZE_INSTRUCTION = """Given a list of tweets, summarize all key facts in **English** regarding {ticker} stock.
Here are some examples:
{examples}
(END OF EXAMPLES)

Tweets:
{tweets}

Facts:"""

In [ ]:
SUMMARIZE_EXAMPLES = """Tweets:
RT @ValaAfshar: Apple has $231.5 billion in cash. It could buy:
Uber
Tesla
Twitter
Airbnb
Netflix
Snapchat
SpaceX
and still have $21 bi…
#Apple announces Q3 2016 revenue of $42.4b: 40.4m iPhones, 9.9m iPads, 4.2m Macs. Read more: https://t.co/wBrFWdTL0M $AAPL
$AAPL Apple China Sales Down ~29% Sequentially, Down 33% YoY
RT @VisualStockRSRC: $AAPL - Apple Earnings Fall on IPhone Slump -- 3rd Update https://t.co/5pSSxlGq8z
RT @DanielEran: Wow: 3/4 contactless payments in US are done via Apple Pay $AAPL
RT @appleinsider: More expensive #iPadPro helps boost $AAPL Q3 revenues despite lower unit sales https://t.co/ZoOi8v93cw https://t.co/AMeR6…
#Apple puts Bob Mansfield in charge of Project Titan car project. Read more: https://t.co/SfPmr2TvPb $AAPL
Apple's market weight rating reiterated at Wells Fargo  &amp; Co.. $120.00 PT. https://t.co/hhQe9eXC1j $AAPL #AAPL
As iPhone sale sag, #Apple touts apps and services instead. Read more: https://t.co/mQK4iZEgCh $AAPL
#Apple granted patent to allow Apple Pencil to work with a Mac's trackpad. Read more: https://t.co/caQmIjfb1L $AAPL
RT @RogerWCheng: Apple @tim_cook says the services business (App Store, Apple Music, etc) will be the size of a Fortune 500 company next ye…
RT @kyurieff: Apple stock soars in after-hours on Q3 beat, pops 5%: https://t.co/MYRmtUIE4J via @TheStreet $AAPL

Facts:
Apple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.
Apple's earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.
Apple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.
Apple's China sales were down around 29% sequentially and 33% YoY.
Despite declining unit sales, Apple's revenue was boosted by more expensive iPad Pro models.
Apple Pay accounted for 3/4 of contactless payments in the US.
Apple's services business (App Store, Apple Music, etc.) was projected to be the size of a Fortune 500 company in the next year.
Apple was reported to be working on a car project called Project Titan, with Bob Mansfield leading it.
The Apple Pencil was granted a patent to work with a Mac's trackpad.
Apple faced declining iPhone sales, but the company focused on promoting apps and services.
The stock price experienced fluctuations after the earnings report, with significant after-hours gains.
Apple's market weight rating was reiterated by Wells Fargo, with a target price of $120.00.

Tweets:
Apple $AAPL revenue down 13%, its first decline in 13 years.
RT @vindugoel: Apple revenue drops 13 percent, ending 13 years of growth. Greater China was especially weak, down 26 percent. $AAPL https:/…
Apple Reports Q2 EPS $1.90 vs. Est. $2.00, Rev. $50.56B vs. Est. $51.97b $AAPL https://t.co/oazu0hpNce
$AAPL Q3 guide: Sales $41B-$43B (vs $47.4B est), GM 37-5%-38% (vs 39.2%) #tech #iPhone #apple
RT @techledes: CEO Tim Cook says it was a "challenging quarter" for $AAPL, which faced "strong macroeconomic headwinds." iPhone, iPad, Macs…
$AAPL #Apple misses on profit and revenue, plans to raise dividend, return $50B more to shareholders https://t.co/AwNq1GY8yr ups #dividend
RT @IGSquawk: $AAPL Apple (Q2 16): Adj EPS $1.90 (est $2.00): Revenue $50.56 bn (est $52.00bn)
Stock down 4.8% JG
RT @USATODAYmoney: Ouch! Apple's earnings of $1.90 a share were well below the $2 the Street expected https://t.co/o6YWFIl4UK $AAPL
RT @usatodaytech: Apple reports first quarterly iPhone sales drop since 2007 debut $AAPL https://t.co/aIuozzuhiP
Apple dividend yield up to 2.3% now. $AAPL

Facts:
Apple reported its Q2 2016 earnings, missing both profit and revenue estimates.
Apple's revenue for the quarter was $50.56 billion, falling short of the estimated $52 billion.
The company's adjusted earnings per share (EPS) was $1.90, lower than the expected $2.00.
This marks the first time in 13 years that Apple experienced a quarterly decline in revenue.
iPhone sales experienced a decline for the first time since its debut in 2007.
The company's guidance for the next quarter indicates expected sales of $41 billion to $43 billion.
Apple's dividend yield increased to 2.3%.
CEO Tim Cook attributed the challenges to strong macroeconomic headwinds, especially in China.
Despite the earnings miss, Apple announced plans to raise its dividend and return $50 billion more to shareholders.
Apple's stock price experienced a decline of around 4.8% in after-hours trading following the earnings report.
"""



In [ ]:

class Summarizer:
    def __init__(self):
        self.summarize_prompt = SUMMARIZE_INSTRUCTION
        self.summarize_examples = SUMMARIZE_EXAMPLES
        self.llm = OpenAILLM()
        self.enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

    def get_summary(self, ticker, tweets):
        summary = None
        if tweets != []:
            prompt = self.summarize_prompt.format(
                                    ticker = ticker,
                                    examples = self.summarize_examples,
                                    tweets = "\n".join(tweets))

            while len(self.enc.encode(prompt)) > 16385:
                tweets = tweets[:-1]
                prompt = self.summarize_prompt.format(
                                        ticker = ticker,
                                        examples = self.summarize_examples,
                                        tweets = "\n".join(tweets))

            summary = self.llm(prompt)

        return summary


    def is_informative(self, summary):
         neg = r'.*[nN]o.*information.*|.*[nN]o.*facts.*|.*[nN]o.*mention.*|.*[nN]o.*tweets.*|.*do not contain.*'
         return not re.match(neg, summary)

In [ ]:



class DataLoader:
    def __init__(self):

        self.price_dir = "/content/sep/data/sample_price/preprocessed/"
        self.tweet_dir = "/content/sep/data/sample_tweet/raw/"
        self.seq_len = 5
        self.summarizer = Summarizer()


    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days)):
            yield start_date + timedelta(n)


    def get_sentiment(self, date_str, price_path):
        price_data = np.genfromtxt(price_path, dtype=str, skip_header=False)

        print(price_data[price_data[:, 0] == date_str][0, 1])
        price_chg = price_data[price_data[:, 0] == date_str][0, 1].astype(float)

        if price_chg > 0.0:
            sentiment = "Positive"
        else:
            sentiment = "Negative"
        return sentiment


    def get_tweets(self, ticker, date_str):
        tweets = []
        tweet_path = os.path.join(self.tweet_dir, ticker, date_str)

        if os.path.exists(tweet_path):
            with open(tweet_path) as f:
                lines = f.readlines()
                for line in lines:
                    tweet_obj = json.loads(line)
                    tweets.append(tweet_obj['text'])
        return tweets


    def load(self, flag):
        data = pd.DataFrame()

        for file in os.listdir(self.price_dir):
            price_path = os.path.join(self.price_dir, file)
            ordered_price_data = np.flip(np.genfromtxt(price_path, dtype=str, skip_header=False), 0)
            ticker = file[:-4]

            tes_idx = round(len(ordered_price_data) * 0.8)
            end_idx = len(ordered_price_data)

            if flag == "train":
                data_range = range(tes_idx)
            else:
                data_range = range(tes_idx, end_idx)

            for idx in data_range:
                summary_all = ""

                end_date_str = ordered_price_data[idx, 0]
                end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
                start_date = end_date - timedelta(days=self.seq_len)
                target = self.get_sentiment(end_date_str, price_path)

                for seq_date in self.daterange(start_date, end_date):
                    seq_date_str = seq_date.strftime("%Y-%m-%d")

                    tweet_data = self.get_tweets(ticker, seq_date_str)
                    summary = "self.summarizer.get_summary(ticker, tweet_data)"

                    if summary and summary is not None and summary != "" and self.summarizer.is_informative(summary):
                        summary_all = summary_all + seq_date_str + "\n" + summary + "\n\n"

                if summary_all != "":
                     row = pd.DataFrame([{'ticker': ticker, 'summary': summary_all.rstrip(), 'target': target}])
                     data = pd.concat([data, row], ignore_index=True)
                     data.to_csv("data.csv", index=False)
        return data


In [ ]:

# Use the assigned values to initialize your DataLoader
dataloader = DataLoader()
data = dataloader.load(flag="train")


['2022-12-01' '2022-11-23' '2022-11-04' '2022-07-07' '2022-05-31'
 '2022-01-03' '2021-12-10' '2021-11-03' '2021-10-15' '2021-10-06'
 '2021-08-23' '2021-07-26' '2021-05-17' '2021-05-14' '2021-03-19'
 '2021-03-05' '2021-02-04' '2020-11-24' '2020-10-23' '2020-10-12'
 '2020-09-15' '2020-09-01' '2020-08-26' '2020-08-25' '2020-08-24'
 '2020-08-05' '2020-08-04' '2020-07-09' '2020-06-16' '2020-06-12'
 '2020-05-14' '2020-04-20' '2020-04-17' '2020-04-14' '2020-04-07']
0.007890
['2022-12-01' '2022-11-23' '2022-11-04' '2022-07-07' '2022-05-31'
 '2022-01-03' '2021-12-10' '2021-11-03' '2021-10-15' '2021-10-06'
 '2021-08-23' '2021-07-26' '2021-05-17' '2021-05-14' '2021-03-19'
 '2021-03-05' '2021-02-04' '2020-11-24' '2020-10-23' '2020-10-12'
 '2020-09-15' '2020-09-01' '2020-08-26' '2020-08-25' '2020-08-24'
 '2020-08-05' '2020-08-04' '2020-07-09' '2020-06-16' '2020-06-12'
 '2020-05-14' '2020-04-20' '2020-04-17' '2020-04-14' '2020-04-07']
0.026775
['2022-12-01' '2022-11-23' '2022-11-04' '2022-07-07' '20

In [ ]:
data = pd.read_csv("data-summery-gpt3-5.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data-summery-gpt3-5.csv'

In [ ]:
""
PREDICT_INSTRUCTION = """Given a list of facts, estimate their overall impact on the price movement of {ticker} stock. Give your response in this format:
(1) Price Movement, which should be either Positive or Negative.
(2) Explanation, which should be in a single, short paragraph.
Here are some examples:
{examples}
(END OF EXAMPLES)

Facts:
{summary}

Price Movement:"""

In [ ]:
PREDICT_EXAMPLES = """Facts:
2016-07-26
Apple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.
Apple's earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.
Apple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.
Apple's China sales were down around 29% sequentially and 33% YoY.
Despite declining unit sales, Apple's revenue was boosted by more expensive iPad Pro models.
Apple Pay accounted for 3/4 of contactless payments in the US.
Apple's services business (App Store, Apple Music, etc.) was projected to be the size of a Fortune 500 company in the next year.
Apple was reported to be working on a car project called Project Titan, with Bob Mansfield leading it.
The Apple Pencil was granted a patent to work with a Mac's trackpad.
Apple faced declining iPhone sales, but the company focused on promoting apps and services.
The stock price experienced fluctuations after the earnings report, with significant after-hours gains.
Apple's market weight rating was reiterated by Wells Fargo, with a target price of $120.00.

Price Movement: Positive

Explanation: Apple reported strong Q3 2016 earnings, surpassing revenue expectations and delivering robust sales figures across its product lines, including iPhones, iPads, and Macs. This performance exceeded market projections and triggered a nearly 5% increase in the stock's after-hours trading. Additionally, Apple's substantial cash reserves of $231.5 billion, capable of facilitating major acquisitions, demonstrated the company's financial stability and growth potential. Despite challenges in China, Apple's diverse revenue sources, including higher-priced iPad Pro models and the dominant Apple Pay in US contactless payments, contributed positively to its overall Price Movement. The promising growth trajectory of Apple's services business added further optimism. While facing declining iPhone sales, Apple's strategic focus on promoting apps and services reflected adaptability in response to changing market dynamics. The consistent support from Wells Fargo with a reiterated market weight rating and target price also reinforced investor confidence. The stock's fluctuations were notable but aligned with the positive earnings report, showcasing the market's responsiveness to Apple's performance.

Facts:
2016-04-26
Apple reported its Q2 2016 earnings, missing both profit and revenue estimates.
Apple's revenue for the quarter was $50.56 billion, falling short of the estimated $52 billion.
The company's adjusted earnings per share (EPS) was $1.90, lower than the expected $2.00.
This marks the first time in 13 years that Apple experienced a quarterly decline in revenue.
iPhone sales experienced a decline for the first time since its debut in 2007.
The company's guidance for the next quarter indicates expected sales of $41 billion to $43 billion.
Apple's dividend yield increased to 2.3%.
CEO Tim Cook attributed the challenges to strong macroeconomic headwinds, especially in China.
Despite the earnings miss, Apple announced plans to raise its dividend and return $50 billion more to shareholders.
Apple's stock price experienced a decline of around 4.8% in after-hours trading following the earnings report.

Price Movement: Negative

Explanation: Apple reported disappointing Q2 2016 earnings, missing both profit and revenue estimates. The company's revenue and adjusted earnings per share fell short of expectations, marking the first quarterly revenue decline in 13 years. iPhone sales, a cornerstone of Apple's business, experienced their first-ever decline since the product's debut in 2007. The weaker-than-expected guidance for the next quarter further dampened investor Price Movement. The CEO's acknowledgment of strong macroeconomic headwinds, particularly in China, indicated external challenges affecting the company's performance. Despite announcing plans to increase dividends and return more to shareholders, the stock price plunged around 4.8% in after-hours trading following the earnings report. Overall, these factors collectively indicate a negative Price Movement surrounding AAPL stock due to its underwhelming financial performance and market outlook.
"""


In [ ]:
class PredictAgent:
    def __init__(self,
                 ticker: str,
                 summary: str,
                 target: str,
                 predict_llm = OpenAILLM()
                 ) -> None:

        self.ticker = ticker
        self.summary = summary
        self.target = target
        self.prediction = ''

        self.predict_prompt = PREDICT_INSTRUCTION
        self.predict_examples = PREDICT_EXAMPLES
        self.llm = predict_llm

        self.__reset_agent()

    def run(self, reset=True) -> None:
        if reset:
            self.__reset_agent()

        facts = "Facts:\n" + self.summary + "\n\nPrice Movement: "
        self.scratchpad += facts
        print(facts, end="")

        self.scratchpad += self.prompt_agent()
        response = self.scratchpad.split('Price Movement: ')[-1]
        self.prediction = response.split()[0]
        print(response, end="\n\n\n\n")

        self.finished = True

    def prompt_agent(self) -> str:
        return self.llm(self._build_agent_prompt())

    def _build_agent_prompt(self) -> str:
        return self.predict_prompt.format(
                            ticker = self.ticker,
                            examples = self.predict_examples,
                            summary = self.summary)

    def is_finished(self) -> bool:
        return self.finished

    def is_correct(self) -> bool:
        return EM(self.target, self.prediction)

    def __reset_agent(self) -> None:
        self.finished = False
        self.scratchpad: str = ''

In [ ]:

REFLECT_INSTRUCTION = """You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to a list of facts to assess their overall impact on the price movement of {ticker} stock. You were unsuccessful in tackling the task because you gave the wrong price movement. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences.

Previous trial:
{scratchpad}

Reflection:"""

In [ ]:
PREDICT_REFLECT_INSTRUCTION = """Given a list of facts, estimate their overall impact on the price movement of {ticker} stock. Give your response in this format:
(1) Price Movement, which should be either Positive or Negative.
(2) Explanation, which should be in a single, short paragraph.
Here are some examples:
{examples}
(END OF EXAMPLES)

{reflections}

Facts:
{summary}

Price Movement:"""

In [ ]:
REFLECTION_HEADER = 'You have attempted to tackle the following task before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly tackling the given task.\n'


In [ ]:
class NShotLLmWhoutReward:
    def __init__(self, model=None, tokenizer=None, num_shots=4):
        self.model = model
        self.tokenizer = tokenizer
        self.num_shots = num_shots

    def __call__(self, prompt):
        query = self.tokenizer.encode(prompt, return_tensors="pt")
        queries = query.repeat((self.num_shots, 1))
        output_ids = self.model.generate(
            queries,
            do_sample=True,
            temperature=0.7,
            max_new_tokens=512,
        )
        output = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        # Since there's no reward model, you can just return the first output
        response = output[0][len(query[0]):]
        response = self.tokenizer.decode(response, skip_special_tokens=True)
        return response


In [ ]:



class PredictReflectAgent(PredictAgent):
    def __init__(self,
                 ticker: str,
                 summary: str,
                 target: str,
                 predict_llm = OpenAILLM(),
                 reflect_llm = OpenAILLM()
                 ) -> None:

        super().__init__(ticker, summary, target, predict_llm)
        self.predict_llm = predict_llm
        self.reflect_llm = reflect_llm
        self.reflect_prompt = REFLECT_INSTRUCTION
        self.agent_prompt = PREDICT_REFLECT_INSTRUCTION
        self.reflections = []
        self.reflections_str: str = ''

    def run(self, reset=True) -> None:
        if self.is_finished() and not self.is_correct():
            self.reflect()

        PredictAgent.run(self, reset=reset)

    def reflect(self) -> None:
        print('Reflecting...\n')
        reflection = self.prompt_reflection()
        self.reflections += [reflection]
        self.reflections_str = format_reflections(self.reflections)
        print(self.reflections_str, end="\n\n\n\n")

    def prompt_reflection(self) -> str:
        return self.reflect_llm(self._build_reflection_prompt())

    def _build_reflection_prompt(self) -> str:
        return self.reflect_prompt.format(
                            ticker = self.ticker,
                            scratchpad = self.scratchpad)
    def run_n_shots_without_reward(self, model, tokenizer, num_shots=2, reset=True) -> None:
        self.llm = NShotLLmWhoutReward(model, tokenizer, num_shots)
        PredictAgent.run(self, reset=reset)

    def _build_agent_prompt(self) -> str:
        prompt = self.agent_prompt.format(
                            ticker = self.ticker,
                            examples = self.predict_examples,
                            reflections = self.reflections_str,
                            summary = self.summary)
        return prompt




def format_reflections(reflections: List[str], header: str = REFLECTION_HEADER) -> str:
    if reflections == []:
        return ''
    else:
        return header + 'Reflections:\n- ' + '\n- '.join([r.strip() for r in reflections])

def EM(prediction, sentiment) -> bool:
    return prediction.lower() == sentiment.lower()

In [ ]:
def summarize_trial(agents):
    correct = [a for a in agents if a.is_correct()]
    incorrect = [a for a in agents if a.is_finished() and not a.is_correct()]
    return correct, incorrect

In [ ]:

agent_cls = PredictReflectAgent

agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]

print("Loaded Train Agents.")

samples = []
for agent in agents:
    agent.run()
    if agent.is_correct():
        prompt = agent._build_agent_prompt()
        response = agent.scratchpad.split('Price Movement: ')[-1]
        sample = {"instruction": prompt, "input": "", "output": response}
        samples.append(sample)

with open('merge_sample.json', 'w') as f:
    json.dump(samples, f, indent=2)



NameError: name 'data' is not defined

In [ ]:
correct, incorrect = summarize_trial(agents)
print(f'Finished Trial 0, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

NameError: name 'agents' is not defined

In [8]:


class SFTDataLoader(object):
    def __init__(self, data, CUTOFF_LEN, VAL_SET_SIZE, tokenizer) -> None:
        super(SFTDataLoader, self).__init__()

        self.data = data
        self.CUTOFF_LEN = CUTOFF_LEN
        self.VAL_SET_SIZE = VAL_SET_SIZE

        self.tokenizer = tokenizer

    def generate_prompt(self, data_point):
        # sorry about the formatting disaster gotta move fast
        if data_point["input"]:
            return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {data_point["instruction"]}

    ### Input:
    {data_point["input"]}

    ### Response:
    {data_point["output"]}"""
        else:
            return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {data_point["instruction"]}

    ### Response:
    {data_point["output"]}"""

    def tokenize(self, prompt):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = self.tokenizer(
            prompt,
            truncation=True,
            max_length=self.CUTOFF_LEN + 1,
            padding="max_length",
        )
        return {
            "input_ids": result["input_ids"][:-1],
            "attention_mask": result["attention_mask"][:-1],
        }

    def generate_and_tokenize_prompt(self, data_point):
        # This function masks out the labels for the input,
        # so that our loss is computed only on the response.
        user_prompt = (
            (
                f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {data_point["instruction"]}

    ### Input:
    {data_point["input"]}

    ### Response:
    """
            )
            if data_point["input"]
            else (
                f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {data_point["instruction"]}

    ### Response:
    """
            )
        )
        len_user_prompt_tokens = (
            len(
                self.tokenizer(
                    user_prompt,
                    truncation=True,
                    max_length=self.CUTOFF_LEN + 1,
                )["input_ids"]
            )
            - 1
        )  # no eos token
        full_tokens = self.tokenizer(
            user_prompt + data_point["output"],
            truncation=True,
            max_length=self.CUTOFF_LEN + 1,
            padding="max_length",
        )["input_ids"][:-1]
        return {
            "input_ids": full_tokens,
            "labels": [-100] * len_user_prompt_tokens
            + full_tokens[len_user_prompt_tokens:],
            "attention_mask": [1] * (len(full_tokens)),
        }

    def load_data(self):
        if self.VAL_SET_SIZE > 0:
            train_val = self.data["train"].train_test_split(
                test_size=self.VAL_SET_SIZE, shuffle=True, seed=42
            )
            train_data = train_val["train"].shuffle().map(
                self.generate_and_tokenize_prompt)
            val_data = train_val["test"].shuffle().map(
                self.generate_and_tokenize_prompt)
        else:
            train_data = self.data["train"].shuffle().map(
                self.generate_and_tokenize_prompt)
            val_data = None

        return train_data, val_data


In [11]:


def supervised_finetune():
    MICRO_BATCH_SIZE = 4
    BATCH_SIZE = 128
    MAX_STEPS = None
    GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
    EPOCHS = 2
    LEARNING_RATE = 3e-4
    CUTOFF_LEN = 256
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.05
    VAL_PCT = 0.1
    TARGET_MODULES = [
        "q_proj",
        "v_proj",
    ]
    DATA_PATH = 'merge_sample.json'
    OUTPUT_DIR = '/content'

    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size

    model = LlamaForCausalLM.from_pretrained(
        'lmsys/vicuna-7b-v1.5-16k',
        load_in_4bit=True,
        device_map=device_map,
    )
    tokenizer = LlamaTokenizer.from_pretrained(
        'lmsys/vicuna-7b-v1.5-16k', add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=TARGET_MODULES,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)
    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
    # tokenizer.padding_side = "left"  # Allow batched inference

    data = load_dataset("json", data_files=DATA_PATH)
    val_set_size = VAL_PCT * len(data)

    now_max_steps = max(
        (len(data["train"]) - val_set_size) // BATCH_SIZE * EPOCHS, EPOCHS)

    MAX_STEPS = now_max_steps


    model.print_trainable_parameters()


    dataloader = SFTDataLoader(
        data, CUTOFF_LEN, val_set_size, tokenizer)
    train_data, val_data = dataloader.load_data()

    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            warmup_steps=100,
            num_train_epochs=EPOCHS,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=20,
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=200 if val_set_size > 0 else None,
            save_steps=200,
            output_dir=OUTPUT_DIR,
            save_total_limit=30,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            report_to= [],
            ignore_data_skip=False,
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(
            tokenizer, mlm=False)
    )
    model.config.use_cache = False

    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
    ).__get__(model, type(model))

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    print("\n If there's a warning about missing keys above, please disregard :)")

    with torch.autocast("cuda"):
        trainer.train()

    model.save_pretrained(OUTPUT_DIR)


In [ ]:
        print("Loading Test Agents...")
        data = dataloader.load(flag="test")




In [ ]:
# data = pd.read_csv("test_summery.csv")

In [ ]:
        # agent_cls = PredictReflectAgent
        # test_agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]
        # print("Loaded Test Agents.")

        # model = LlamaForCausalLM.from_pretrained(
        # 'lmsys/vicuna-7b-v1.5-16k',
        # load_in_4bit=True,
        # device_map="auto",
        #   )
        # tokenizer = LlamaTokenizer.from_pretrained(
        # 'lmsys/vicuna-7b-v1.5-16k', add_eos_token=True
        #      )


Loaded Test Agents.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
# for agent in test_agents:
#             agent.run_n_shots_without_reward(
#                               model=model,
#                               tokenizer=tokenizer,

#                               )
# correct, incorrect = summarize_trial(test_agents)
# print(f'Finished evaluation, Correct: {len(correct)}, Incorrect: {len(incorrect)}')


Facts:
2021-12-05
BABA stock has experienced a significant drop from $320 to $110.
Investors are more focused on trading the stock than determining its actual value amidst delisting concerns.
JP Morgan values Alibaba at 0.3x of 2022E core ecommerce profit.
Chinese regulators have had constructive communication with US regulators.
The stock has been active in trading, with reports of positive news.
There is increased trading activity and interest in BABA stock, with a high volume of call options being traded.
There is notable interest and discussion around BABA among WallstreetBets and Reddit users.
Some analysts believe that the sell-off of Chinese internet stocks, like BABA, is overdone.
There have been discussions around the use of OTC trading in relation to various stocks, including BABA.
There are comments and speculations about the potential rise of BABA stock and its performance in the market.

2021-12-06
Based on the provided tweets, here are some key facts regarding BABA stock:

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


ValueError: invalid literal for int() with base 10: 'c'

In [9]:
ls

adapter_config.json  adapter_model.bin  drive/  merge_sample.json  sample_data/  sep/


In [12]:
supervised_finetune()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



 If there's a warning about missing keys above, please disregard :)


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True 

Step,Training Loss,Validation Loss


In [3]:

    model_name='/content'
    output_name='/content/merge'
    peft_model_id = model_name
    peft_config = PeftConfig.from_pretrained(peft_model_id)
    print("peft_config: ", peft_config)

    model = AutoModelForCausalLM.from_pretrained(
         peft_config.base_model_name_or_path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        load_in_8bit=True,
        device_map="auto",
        offload_folder="offload",
        offload_state_dict=True)



      # Load the Lora model
    model = PeftModel.from_pretrained(model, peft_model_id)
    model.eval()

    model = model.merge_and_unload()

    if output_name is None:
        output_name = f"{model_name}-adapter-merged"
        model.save_pretrained(output_name)
    else:
        model.save_pretrained(f"{output_name}")



peft_config:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='lmsys/vicuna-7b-v1.5-16k', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=8, target_modules={'q_proj', 'v_proj'}, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:67: UserWarn

In [ ]:

def remove_reflections(prompt: str) -> str:
    prefix = prompt.split('You have attempted to tackle the following task before and failed.')[0]
    suffix = prompt.split('\n\nFacts:')[-1]
    return prefix.strip('\n').strip() + '\n\nFacts' +  suffix.strip('\n').strip()

In [ ]:
        # Collect comparison data
        comparison_data = []

        for trial in range(2):
            for idx, agent in enumerate([a for a in agents if not a.is_correct()]):

                prev_response = agent.scratchpad.split('Price Movement: ')[-1]
                agent.run()

                if agent.is_correct():
                    print("-------------" ,agent._build_agent_prompt(), "\n\n\n")
                    prompt = remove_reflections(agent._build_agent_prompt())
                    response = agent.scratchpad.split('Price Movement: ')[-1]

                    sample = {"user_input": prompt, "completion_a": prev_response, "completion_b": response}
                    comparison_data.append(sample)

            correct, incorrect = summarize_trial(agents)
            print(f'Finished Trial {trial+1}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

In [ ]:
comparison_data

In [ ]:
        os.makedirs('/content', exist_ok=True)
        comparison_data_path = os.path.join('/content', "comparison_data.json")

        if comparison_data:
            with open(comparison_data_path, 'w') as f:
                f.write(json.dumps(comparison_data))

In [ ]:

class RewardDataLoader(object):
    def __init__(self, dataset_name, train_subset, eval_subset, num_proc, tokenizer) -> None:
        super(RewardDataLoader, self).__init__()

        self.dataset_name = dataset_name

        self.train_subset = train_subset
        self.eval_subset = eval_subset

        self.num_proc = num_proc
        self.tokenizer = tokenizer

    # Turn the dataset into pairs of post + summaries, where text_j is the preferred question + answer and text_k is the other.
    # Then tokenize the dataset.

    def preprocess_function(self, examples):
        new_examples = {
            "input_ids_j": [],
            "attention_mask_j": [],
            "input_ids_k": [],
            "attention_mask_k": [],
        }
        # for question, response_j, response_k in zip(examples["question"], examples["response_j"], examples["response_k"]):
        for question, response_j, response_k in zip(examples["user_input"], examples["completion_a"], examples["completion_b"]):
            tokenized_j = self.tokenizer(
                "Question: " + question + "\n\nAnswer: " + response_j, truncation=True)
            tokenized_k = self.tokenizer(
                "Question: " + question + "\n\nAnswer: " + response_k, truncation=True)

            new_examples["input_ids_j"].append(tokenized_j["input_ids"])
            new_examples["attention_mask_j"].append(
                tokenized_j["attention_mask"])
            new_examples["input_ids_k"].append(tokenized_k["input_ids"])
            new_examples["attention_mask_k"].append(
                tokenized_k["attention_mask"])

        return new_examples

    def load_data(self):

        # Load the human stack-exchange-paired dataset for tuning the reward model.
        # train_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/reward", split="train")
        train_dataset = load_dataset(self.dataset_name, split="train")
        if self.train_subset > 0:
            train_dataset = train_dataset.select(range(min(len(train_dataset), self.train_subset)))
        # eval_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/evaluation", split="train")
        eval_dataset = load_dataset(self.dataset_name, split="train")
        if self.eval_subset > 0:
            eval_dataset = eval_dataset.select(range(min(len(eval_dataset), self.eval_subset)))

        original_columns = train_dataset.column_names

        # preprocess the dataset and filter out QAs that are longer than 512
        print("train_dataset: ", len(train_dataset))
        train_dataset = train_dataset.map(
            self.preprocess_function, batched=True, num_proc=self.num_proc, remove_columns=original_columns
        )
        # train_dataset = train_dataset.filter(lambda x: len(
        #     x["input_ids_j"]) <= 512 and len(x["input_ids_k"]) <= 512)
        print("train_dataset: ", len(train_dataset))

        print("eval_dataset: ", len(eval_dataset))
        eval_dataset = eval_dataset.map(
            self.preprocess_function, batched=True, num_proc=self.num_proc, remove_columns=original_columns)
        # eval_dataset = eval_dataset.filter(lambda x: len(
        #     x["input_ids_j"]) <= 512 and len(x["input_ids_k"]) <= 512)
        print("eval_dataset: ", len(eval_dataset))

        return train_dataset, eval_dataset


In [ ]:
ls

database/  sample_data/


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:



# DEFAULT_PAD_TOKEN = "[PAD]"
# DEFAULT_EOS_TOKEN = "</s>"
# DEFAULT_BOS_TOKEN = "</s>"
# DEFAULT_UNK_TOKEN = "</s>"


def train_reward_model():


    dataset_name = "./database/"
    print("dataset_name: ", dataset_name)

    # Define the training args. Needs to be done before the model is loaded if you are using deepspeed.
    # model_name_split = script_args.reward_base_model.split("/")[-1]
    # output_name = (
    # f"{model_name_split}_peft_stack-exchange-paired_rmts__{script_args.train_subset}_{script_args.reward_learning_rate}"
    # )
    output_name = "reward_adapter"

    # Define the training args. Needs to be done before the model is loaded if you are using deepspeed.
    training_args = TrainingArguments(
        output_dir=output_name,
        learning_rate=2e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        num_train_epochs=1,
        weight_decay=0.001,
        evaluation_strategy="steps",
        eval_steps=200,  # 500,
        save_strategy="steps",
        save_steps=200,  # 500,
        save_total_limit=2,
        gradient_accumulation_steps=32,
        gradient_checkpointing=True,
        deepspeed=None,
        # local_rank=script_args.local_rank,
        remove_unused_columns=False,
        label_names=[],

        # bf16=script_args.bf16,
        # fp16=True,
        logging_strategy="steps",
        logging_steps=10,
        optim="adamw_hf",
        lr_scheduler_type="linear",
        report_to="none"
    )
    reward_base_model = "gpt2-xl"
    # Load the value-head model and tokenizer.
    if "llama" in reward_base_model or "vicuna" in reward_base_model or "Vicuna" in reward_base_model:
        tokenizer = LlamaTokenizer.from_pretrained(reward_base_model,dtype=torch.float16)
        config = LlamaConfig.from_pretrained(reward_base_model)
    else:
        tokenizer = AutoTokenizer.from_pretrained(reward_base_model, trust_remote_code=True)
        config = AutoConfig.from_pretrained(reward_base_model, trust_remote_code=True)

    # if "llama" in script_args.reward_base_model or "vicuna" in script_args.reward_base_model or "Vicuna" in script_args.reward_base_model:
    #     # required for llama
    #     tokenizer.add_special_tokens(
    #         {
    #             "eos_token": DEFAULT_EOS_TOKEN,
    #             "bos_token": DEFAULT_BOS_TOKEN,
    #             "unk_token": DEFAULT_UNK_TOKEN,
    #             "pad_token": DEFAULT_PAD_TOKEN,
    #         }
    #     )
    # else:
    #     # required for gpt2
    #     tokenizer.pad_token = tokenizer.eos_token


    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    print("device_map: ", device_map)
    # model = AutoModelForSequenceClassification.from_pretrained(
    #    script_args.reward_base_model, num_labels=1, torch_dtype=torch.bfloat16
    # )

    if "llama" in reward_base_model or "vicuna" in reward_base_model or "Vicuna" in reward_base_model:
        model = LlamaForSequenceClassification.from_pretrained(
            reward_base_model,
            num_labels=1,
            load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map=device_map,
        )
    else:
        model = AutoModelForSequenceClassification.from_pretrained(
            reward_base_model,
            num_labels=1,
            load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map=device_map,
            trust_remote_code=True,
        )

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=8,
        lora_alpha=16,  # 32,
        lora_dropout=0.05,  # 0.1,
        bias="none",
    )

    model = get_peft_model(model, peft_config)

    model.print_trainable_parameters()

    # Need to do this for gpt2, because it doesn't have an official pad token.
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    model.config.use_cache = not False
    num_proc = 1  # Can adjust to be higher if you have more processors.


    reward_dataloder = RewardDataLoader(dataset_name, 100000, 50000, num_proc, tokenizer)
    train_dataset, eval_dataset = reward_dataloder.load_data()

    @dataclass
    class RewardDataCollatorWithPadding:
        tokenizer: PreTrainedTokenizerBase
        padding: Union[bool, str, PaddingStrategy] = True
        max_length: Optional[int] = 256
        pad_to_multiple_of: Optional[int] = None
        return_tensors: str = "pt"

        def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
            features_j = []
            features_k = []
            for feature in features:
                features_j.append(
                    {
                        "input_ids": feature["input_ids_j"],
                        "attention_mask": feature["attention_mask_j"],
                    }
                )
                features_k.append(
                    {
                        "input_ids": feature["input_ids_k"],
                        "attention_mask": feature["attention_mask_k"],
                    }
                )
            batch_j = self.tokenizer.pad(
                features_j,
                padding=self.padding,
                max_length=self.max_length,
                pad_to_multiple_of=self.pad_to_multiple_of,
                return_tensors=self.return_tensors,
            )
            batch_k = self.tokenizer.pad(
                features_k,
                padding=self.padding,
                max_length=self.max_length,
                pad_to_multiple_of=self.pad_to_multiple_of,
                return_tensors=self.return_tensors,
            )
            batch = {
                "input_ids_j": batch_j["input_ids"],
                "attention_mask_j": batch_j["attention_mask"],
                "input_ids_k": batch_k["input_ids"],
                "attention_mask_k": batch_k["attention_mask"],
                "return_loss": True,
            }
            return batch


    # Define the metric that we'll use for validation.
    accuracy = evaluate.load("accuracy")


    def compute_metrics(eval_pred):
        predictions, _ = eval_pred
        # Here, predictions is rewards_j and rewards_k.
        # We want to see how much of the time rewards_j > rewards_k.
        predictions = np.argmax(predictions, axis=0)
        labels = np.zeros(predictions.shape)
        return accuracy.compute(predictions=predictions, references=labels)


    class RewardTrainer(Trainer):
        # Define how to compute the reward loss. We use the InstructGPT pairwise logloss: https://arxiv.org/abs/2203.02155
        def compute_loss(self, model, inputs, return_outputs=False):
            rewards_j = model(
                input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
            rewards_k = model(
                input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
            loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
            if return_outputs:
                return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
            return loss


    # Train the model, woohoo.
    trainer = RewardTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        data_collator=RewardDataCollatorWithPadding(
            tokenizer=tokenizer, max_length=512, pad_to_multiple_of=8),
    )

    model.config.use_cache = False

    trainer.train(False)

    print("Saving last checkpoint of the model")
    # model.save_pretrained(output_name + "_peft_last_checkpoint")
    model.save_pretrained(output_name)


In [ ]:
train_reward_model()

dataset_name:  ./database/


device_map:  auto


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-xl and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,459,200 || all params: 1,560,072,000 || trainable%: 0.15763375023716855
train_dataset:  4


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

eval_dataset:  4


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2624: U

Step,Training Loss,Validation Loss


Saving last checkpoint of the model


In [2]:

    model_name='/content'
    output_name='/content/merge'
    peft_model_id = model_name
    peft_config = PeftConfig.from_pretrained(peft_model_id)
    print("peft_config: ", peft_config)





peft_config:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='gpt2-xl', revision=None, task_type='SEQ_CLS', inference_mode=True, r=8, target_modules={'c_attn'}, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})


In [3]:
    model = AutoModelForCausalLM.from_pretrained(
         peft_config.base_model_name_or_path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        load_in_8bit=True,
        device_map="auto",
        offload_folder="offload",
        offload_state_dict=True)




/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [11]:
ls

adapter_config.json  adapter_model.bin  sample_data/  sep/


In [4]:

    # Load the Lora model
    model = PeftModel.from_pretrained(model, peft_model_id)


In [6]:
    model.eval()

    model = model.merge_and_unload()

    if output_name is None:
        output_name = f"{model_name}-adapter-merged"
        model.save_pretrained(output_name)
    else:
        model.save_pretrained(f"{output_name}")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:67: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


# **last part**

In [4]:
    reward_model_name = "/content/drive/MyDrive/rl_merge"

    # dataset_name = "lvwerra/stack-exchange-paired"
    dataset_name = "/content/drive/MyDrive/database"
    print("dataset_name: ", dataset_name)

    config = PPOConfig(
        model_name="/content/drive/MyDrive/finetune_merge",
        learning_rate=1.4e-5,
        log_with=None,
        batch_size=1,
        mini_batch_size=1,
        gradient_accumulation_steps=1,
        optimize_cuda_cache=True,
        early_stopping=True,
        target_kl=0.1,
        ppo_epochs=4,
        seed=0,
    )

    # train_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/rl", split="train")
    # train_dataset = train_dataset.select(range(100000))
    train_dataset = load_dataset(dataset_name, split="train")
    # train_dataset = train_dataset.select(range(100))
    # We then define the arguments to pass to the sentiment analysis pipeline.
    # We set `return_all_scores` to True to get the sentiment score for each token.
    # sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16, "truncation": True}
    sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 1, "truncation": True}

    # tokenizer = AutoTokenizer.from_pretrained(script_args.tokenizer_name)
    # GPT-2 tokenizer has a pad token, but it is not eos_token by default. We need to set it to eos_token.
    # only for this model.

    # if "llama" in script_args.tokenizer_name or "vicuna" in script_args.rl_base_model or "Vicuna" in script_args.rl_base_model:
    tokenizer = LlamaTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5-16k")
    # else:
    #     tokenizer = AutoTokenizer.from_pretrained(script_args.tokenizer_name)


    # if "llama" in script_args.tokenizer_name or "vicuna" in script_args.rl_base_model or "Vicuna" in script_args.rl_base_model:
    #     tokenizer.add_special_tokens(
    #         {
    #             "eos_token": DEFAULT_EOS_TOKEN,
    #             "bos_token": DEFAULT_BOS_TOKEN,
    #             "unk_token": DEFAULT_UNK_TOKEN,
    #             "pad_token": DEFAULT_PAD_TOKEN,
    #         }
    #     )
    # else:
    #     tokenizer.pad_token = tokenizer.eos_token


    # Below is an example function to build the dataset. In our case, we use the IMDB dataset
    # from the `datasets` library. One should customize this function to train the model on
    # its own dataset.

dataset_name:  /content/drive/MyDrive/database


In [5]:
    def build_dataset(
        tokenizer, dataset_name="lvwerra/stack-exchange-paired", input_min_text_length=2, input_max_text_length=8
    ):
        """
        Build dataset for training. This builds the dataset from `load_dataset`, one should
        customize this function to train the model on its own dataset.

        Args:
            dataset_name (`str`):
                The name of the dataset to be loaded.

        Returns:
            dataloader (`torch.utils.data.DataLoader`):
                The dataloader for the dataset.
        """

        # load imdb with datasets
        # ds = load_dataset(dataset_name, data_dir="data/rl", split="train")
        ds = load_dataset(dataset_name, split="train")
        original_columns = ds.column_names
        num_proc = 1 #24

        def preprocess_function(examples):
            new_examples = {
                "query": [],
                "input_ids": [],
            }
            # for question in examples["question"]:
            for question in examples["user_input"]:
                query = "Question: " + question + "\n\nAnswer: "
                tokenized_question = tokenizer(query, truncation=True)
                new_examples["query"].append(query)
                new_examples["input_ids"].append(tokenized_question["input_ids"])

            return new_examples

        ds = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=num_proc,
            remove_columns=original_columns,
        )
        # ds = ds.filter(lambda x: len(x["input_ids"]) < 512, batched=False)

        ds.set_format(type="torch")
        return ds


    # We retrieve the dataloader by calling the `build_dataset` function.
    # dataset = build_dataset(tokenizer)
    dataset = build_dataset(tokenizer, dataset_name=dataset_name)

In [ ]:
    def collator(data):
        return dict((key, [d[key] for d in data]) for key in data[0])


    # # set seed before initializing value head for deterministic eval
    # set_seed(config.seed)

    # Now let's build the model, the reference model, and the tokenizer.
    current_device = Accelerator().local_process_index

    lora_config = LoraConfig(
        r=8, #16,
        lora_alpha=16, #32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        config.model_name,
        load_in_4bit=True,
        # device_map={"": current_device},
        device_map="auto",
        peft_config=lora_config,
        # layer_norm_names=[],
        # torch_dtype=torch.float16,
        quantization_config=BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)
    )
    print("finetune model: ", config.model_name, type(model))
    print("finetune model's is_loaded_in_4bit: ", model.is_loaded_in_4bit)

In [ ]:

    optimizer = None
    # if script_args.adafactor:
    #     optimizer = Adafactor(
    #         filter(lambda p: p.requires_grad, model.parameters()),
    #         scale_parameter=False,
    #         relative_step=False,
    #         warmup_init=False,
    #         lr=config.learning_rate,
    #     )
    # We then build the PPOTrainer, passing the model, the reference model, the tokenizer
    ppo_trainer = PPOTrainer(
        config,
        model,
        ref_model=None,
        tokenizer=tokenizer,
        dataset=dataset,
        data_collator=collator,
        optimizer=optimizer,
    )

    # We then build the sentiment analysis pipeline, passing the model name and the
    # sentiment analysis pipeline arguments. Let's also make sure to set the device
    # to the same device as the PPOTrainer.
    device = ppo_trainer.accelerator.device
    if ppo_trainer.accelerator.num_processes == 1:
        device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a ` pipeline` bug
    print("device: ", device)


    print("reward_model_name: ", reward_model_name)
    #! my self code to try peft reward model
    # reward_model = LlamaForSequenceClassification.from_pretrained(
    #     reward_model_name,
    #     load_in_4bit=True,
    #     device_map="auto",
    #     torch_dtype=torch.float16,
    # )
    # print("reward_model: ", type(reward_model))
    # print("reward_model is_loaded_in_4bit: ", reward_model.is_loaded_in_4bit)

    # reward_model = prepare_model_for_int8_training(reward_model)
    # reward_model_config = LlamaConfig.from_pretrained(reward_model_name)

    sentiment_pipe = pipeline(
        "sentiment-analysis",
        model=reward_model_name,
        # model=reward_model,
        # config=reward_model_config,
        device_map="auto",
        # TypeError: LlamaForSequenceClassification.__init__() got an unexpected keyword argument 'peft_config'
        model_kwargs={"load_in_4bit": True},
        tokenizer=tokenizer,
        # torch_dtype=torch.float16,
    )

    # We then define the arguments to pass to the `generate` function. These arguments
    # are passed to the `generate` function of the PPOTrainer, which is a wrapper around
    # the `generate` function of the trained model.
    generation_kwargs = {
        # "min_length": -1,
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": 100_000,
    }
    output_min_length = 32
    output_max_length = 128
    output_length_sampler = LengthSampler(output_min_length, output_max_length)

    for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
        question_tensors = batch["input_ids"]

        response_tensors = ppo_trainer.generate(
            question_tensors,
            return_prompt=False,
            length_sampler=output_length_sampler,
            **generation_kwargs,
        )
        output_dir="/content/final"
        batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

        # Compute sentiment score
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        print(epoch,texts,"------")
        pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
        print(pipe_outputs,"------")
        rewards = [torch.tensor(output[0]["score"]) for output in pipe_outputs]

        # Run PPO step
        stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)



        ppo_trainer.save_pretrained(output_dir + "step_saved")
